In [1]:
import os
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import rasterio as rio
import geopandas as gp

import fiona 
import rasterio
import rasterio.mask

import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import matplotlib.dates as mdates

from matplotlib.lines import Line2D
from matplotlib.ticker import StrMethodFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm
from PyIF import te_compute as te
from sklearn import metrics

from osgeo import gdal, osr, ogr
from tqdm import tqdm
from scipy import stats, spatial, signal, fftpack
from scipy.optimize import curve_fit


/Users/aakashahamed/anaconda3/envs/rsenv/lib/python3.8/site-packages/numba/core/decorators.py:255: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [2]:
gdf = gp.read_file("../shape/sierra_catchments.shp")

In [3]:

def get_fnf(stn_id):
    '''
    Query CA DWR website to get reservoir storage for an area of interest
    '''
    print("**** Fetching FNF for {} ****".format(stn_id))

    url = "https://cdec.water.ca.gov/dynamicapp/req/CSVDataServlet?Stations={}&SensorNums=8&dur_code=D&Start=2000-09-01&End=2021-09-01".format(stn_id)
    df = pd.read_csv(url)

    df[stid] = pd.to_numeric(df['VALUE'], errors='coerce').interpolate(how = 'linear') * 0.0283168 # cfs --> cms 
    df.index = pd.to_datetime(df['DATE TIME'])
    df.index.names = ['date']
    df.drop(['STATION_ID', "VALUE", "DURATION", "SENSOR_NUMBER", 
             "SENSOR_TYPE", "OBS DATE",'DATE TIME', "DATA_FLAG", "UNITS"], axis = 1, inplace = True)

    df[df[stid] < 0] = np.nan
    return df#.interpolate(how = 'polynomial', order = 3)

In [4]:
# Set up outlists 
results_df = []
param_df = []
summary_df = []

# Loop through catchments 
for stid in list(gdf['stid'])[:]:
        
    # get area
    akm2 = float(gdf[gdf['stid'] == stid].area_km2)
    fnf_df = get_fnf(stid)

    hdf = fnf_df.copy()

    hdf.columns = ['y']
    hdf['y'] = hdf['y'] / float(akm2) * 1e-9 * 1e6 #
    hdf['dy'] = hdf['y'].diff()

    hdf = hdf.dropna()

    # Drop rows with increasing baseflow 
    droprows = []
    for idx, row in hdf.reset_index().iterrows():
        if row['dy'] > 0:
            droprows.append([idx + x for x in list(range(-1,3))])

    drs = [item for sublist in droprows for item in sublist]
    drs = [x for x in drs if x>=0 and x <len(hdf)]

    odf = hdf.reset_index().drop(hdf.reset_index().index[drs]).set_index(['date'])

    # Remove vvv small values 
    odf = odf[(np.abs(odf) > 1e-10).all(axis=1)]

    # Remove outliers 
    odf = odf[(np.abs(stats.zscore(odf['y'])) < 3)]

    # keep only summer 
    odf['midx'] = odf.index.month
    odf[odf['midx'].isin([6,7,8,9,])]

    odf.dropna(inplace = True)
    x = odf['y'].values
    y = odf['dy'].abs().values

    def f(x, a, b):
        return np.log(a*x**b)

    popt,pcov=curve_fit(f, x, np.log(y))
    
    temp_df = pd.DataFrame([x,y]).T
    temp_df.columns = ['y','dy']
    temp_df['id'] = [stid for x in range(len(temp_df))]
    results_df.append(temp_df)
    
    a, b = popt
    print("Parameters: a={},  b={}, K = {}".format(a,b, abs(b/(1-b))))
    
    #parameter form curve_fit
    py=a*x**b
    
    # Read rainfall and snowmelt data
    smlt_fn_1d = "../data/Watersheds/{}_smlt.npy".format(stid)
    prcp_fn_1d = "../data/Watersheds/1d/{}_1d_prcp.npy".format(stid)
    et_fn_1d = "../data/Watersheds/et/{}_et.npy".format(stid)
    
    # Read runoff
    bf = pd.read_csv("../data/baseflow_sep/baseflow_mm.csv")
    bf['date'] = pd.to_datetime(bf['date'])
    bf.set_index("date", inplace = True)    
    sr = pd.read_csv("../data/baseflow_sep/surface_runoff_mm.csv") 
    sr['date'] = pd.to_datetime(sr['date'])
    sr.set_index("date", inplace = True)   

    # Load files as np arrays
    smlt = np.load(smlt_fn_1d)
    smlt = smlt*24 # apply scaling factor 
    prcp = np.load(prcp_fn_1d)
#     et_all = np.load(et_fn_1d)

    # filter the runoff data to select watershed
    sr_df = pd.DataFrame(sr[stid].interpolate(how = 'linear'))[:-1] / float(akm2) * 1e-9 * 1e6 * 0.0283168
    bf_df = pd.DataFrame(bf[stid].interpolate(how = 'linear'))[:-1] / float(akm2) * 1e-9 * 1e6 * 0.0283168

    # filter the et data to coincide with snodas data 
#     et_idx = pd.date_range("2001-01-01",'2021-12-31')
#     diff_start = bf_df.index[0] - et_idx[0]
#     ndays_start = diff_start.days
#     diff_end = bf_df.index[-1] - et_idx[-1]
#     ndays_end = diff_end.days
#     et = et_all[:,:,ndays_start:ndays_end]
#     et = np.where(et==9.999,np.nan, et) # mask nodata vals 
    
    smlt_mean = [np.nanmean(smlt[:,:,x]) for x in range(0,smlt.shape[2])]
    prcp_mean = [np.nanmean(prcp[:,:,x]) for x in range(0,prcp.shape[2])]
#     et_mean = [np.nanmean(et[:,:,x]) for x in range(0,et.shape[2])]

    ann_pmean = np.nanmean(prcp_mean) * 365
    ann_smean = np.nanmean(smlt_mean) * 365
    ann_bf_mean = float((bf_df.mean().values * 86400)) * 365
    
    pdf = pd.DataFrame([stid, ann_pmean, ann_smean, ann_bf_mean, abs(b/(1-b))]).T
    pdf.columns = ['id', 'Rainfall (mm)' , 'Snowmelt (mm)', "Baseflow (mm)", "K"]
    param_df.append(pdf)


**** Fetching FNF for SJF ****


<ipython-input-4-8731f3a81f3b>:47: RuntimeWarning: invalid value encountered in log
  return np.log(a*x**b)


Parameters: a=0.05927495454128906,  b=0.9667269994646366, K = 29.05439797763877


<ipython-input-4-8731f3a81f3b>:94: RuntimeWarning: Mean of empty slice
  smlt_mean = [np.nanmean(smlt[:,:,x]) for x in range(0,smlt.shape[2])]


**** Fetching FNF for TLG ****


...

<ipython-input-4-8731f3a81f3b>:47: RuntimeWarning: invalid value encountered in log
  return np.log(a*x**b)


Parameters: a=0.06933687826651265,  b=0.9884795883858348, K = 85.80245406947338


<ipython-input-4-8731f3a81f3b>:94: RuntimeWarning: Mean of empty slice
  smlt_mean = [np.nanmean(smlt[:,:,x]) for x in range(0,smlt.shape[2])]


**** Fetching FNF for SHA ****
Parameters: a=227.03899491415518,  b=1.7208093974027086, K = 2.3873293045336235


<ipython-input-4-8731f3a81f3b>:94: RuntimeWarning: Mean of empty slice
  smlt_mean = [np.nanmean(smlt[:,:,x]) for x in range(0,smlt.shape[2])]


In [5]:
pd.concat(param_df)

,id,Rainfall (mm),Snowmelt (mm),Baseflow (mm),K
0,SJF,542.444552,70.142832,206.225574,29.054398
0,TLG,537.434952,91.369415,215.431409,5.827257
0,ISB,294.7725,60.778152,80.320773,11.890671
0,NAT,394.488417,89.736513,252.820745,5.765484
0,MHB,170.043693,73.546216,117.550204,133.101392
0,NML,560.171487,91.134414,192.056863,4.358234
0,ORO,340.638435,74.0447,221.460364,15.167482
0,TRM,363.959396,63.842251,159.401784,65.561954
0,MKM,508.605363,80.156724,239.486352,1.65572
0,SCC,132.863159,58.318631,65.638448,1.696909


In [6]:
t1df = pd.merge(gdf,pd.concat(param_df), left_on = 'stid', right_on = 'id')

In [7]:
t1df['K'] = np.ceil(t1df['K'])

In [8]:
t1df

,outlet_id,area_km2,catch_name,stid,elev_ft,agency,geometry,id,Rainfall (mm),Snowmelt (mm),Baseflow (mm),K
0,SAN JOAQUIN RIVER BELOW FRIANT,5122.5534,SAN JOAQUIN R,SJF,294,US Geological Survey,"POLYGON ((-119.17083 37.73750, -119.17000 37.7...",SJF,542.444552,70.142832,206.225574,30
1,TUOLUMNE R-LA GRANGE DAM,4722.4053,TUOLUMNE R,TLG,170,Turlock Irrigation District,"POLYGON ((-119.65583 38.22833, -119.64917 38.2...",TLG,537.434952,91.369415,215.431409,6
2,ISABELLA DAM,6226.9722,KERN R,ISB,2635,US Army Corps of Engineers,"POLYGON ((-118.40750 36.70083, -118.40500 36.7...",ISB,294.7725,60.778152,80.320773,12
3,LAKE NATOMA (NIMBUS DAM),5838.2694,AMERICAN R,NAT,132,US Bureau of Reclamation,"POLYGON ((-120.60083 39.32083, -120.59833 39.3...",NAT,394.488417,89.736513,252.820745,6
4,COSUMNES RIVER AT MICHIGAN BAR,1672.7634,COSUMNES R,MHB,168,US Geological Survey,"POLYGON ((-120.55417 38.75750, -120.55167 38.7...",MHB,170.043693,73.546216,117.550204,134
5,NEW MELONES RESERVOIR,2801.7900,STANISLAUS R,NML,1135,US Bureau of Reclamation,"POLYGON ((-119.92417 38.52000, -119.92167 38.5...",NML,560.171487,91.134414,192.056863,5
6,OROVILLE DAM,11475.1485,FEATHER R,ORO,900,CA Dept of Water Resources/O&M Oroville Field ...,"POLYGON ((-121.22667 40.50083, -121.22333 40.5...",ORO,340.638435,74.0447,221.460364,16
7,TERMINUS DAM,1701.7128,KAWEAH R,TRM,752,US Army Corps of Engineers,"POLYGON ((-118.93333 36.72667, -118.93167 36.7...",TRM,363.959396,63.842251,159.401784,66
8,MOKELUMNE-MOKELUMNE HILL,1688.1372,MOKELUMNE R,MKM,575,East Bay Municipal Utility District,"POLYGON ((-120.01583 38.66583, -120.01500 38.6...",MKM,508.605363,80.156724,239.486352,2
9,SUCCESS DAM,1182.0492,TULE R,SCC,692,US Army Corps of Engineers,"POLYGON ((-118.68083 36.33417, -118.67833 36.3...",SCC,132.863159,58.318631,65.638448,2


In [9]:
t1df.drop(['stid','agency','outlet_id','geometry'], axis =1).to_csv("../data/Table1.csv")